In [ ]:
#need pip install spotipy to run this
import json
import spotipy
import csv
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import base64
import itertools
import seaborn as sns
import scipy.stats as stats
from scipy.stats import ttest_ind
from requests import post, get
from config import cid, secret

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
results = sp.search(q='artist:' + 'Drake', type='artist')

In [ ]:
auth_string = cid + ":" + secret
auth_bytes = auth_string.encode('utf-8')
auth_base64 = str(base64.b64encode(auth_bytes), 'utf-8')
url = "https://accounts.spotify.com/api/token"
headers = {
    'Authorization': 'Basic ' + auth_base64,
    'Content-Type': 'application/x-www-form-urlencoded'
}
data = {"grant_type": "client_credentials"}
result = post(url, headers=headers, data=data)
json_result = json.loads(result.content)
token = json_result['access_token']
print(token)

In [ ]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZF1DX7Jl5KP2eZaS?si=fa88101bc2b04d96"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [ ]:
sp.playlist_tracks(playlist_URI)['items'][0]['track']['duration_ms']

In [ ]:
genres = ''
song_name = []
artist_name = []
artist_pop = []
genre_list = []
album = []
track_pop = []
years = []
links = []
song_years = []
song_duration = []
genres_compiled = []
counts = []
#playlist_df = pd.read_csv('Resources/Playlistlinks.csv')
with open('Resources/Playlistlinks.csv') as file_obj:
    reader_obj = csv.reader(file_obj)
    next(reader_obj)
    for row in reader_obj:
        years.append(row[0])
        links.append(row[1])
for (link, year) in zip(links, years):
    #URI
    playlist_link = link
    playlist_URI = playlist_link.split("/")[-1].split("?")[0]
    #track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]
    print(f"Starting link {link}")
    count = 1
    for track in sp.playlist_tracks(playlist_URI)["items"]:
        if count == 51:
            break
        track_uri = track["track"]["uri"]
        #print(track_uri)
        #track = sp.track(track_uri)
        #print(track)
        #break
        song_genre = sp.track(track_uri)
        #print(song_genre['genre'])

        #Track name
        song_name.append(track["track"]["name"])

        #Main Artist
        artist_uri = track["track"]["artists"][0]["uri"]
        #print(f"Uri:{artist_uri}")
        artist_info = sp.artist(artist_uri)

        #Name, popularity, genre
        artist_name.append(track["track"]["artists"][0]["name"])
        artist_pop.append(artist_info["popularity"])
        artist_genres = artist_info["genres"]
        for genre in artist_genres:
            if genre == artist_genres[-1]:
                genres = genres + genre + ' '
            else: 
                genres = genres + genre + ', '
        genre_list.append(genres)
        genres = ''
        #Album
        album.append(track["track"]["album"]["name"])

        song_duration.append((track['track']['duration_ms']) / 60000)

        song_years.append(year)
    
        #Popularity of the track
        track_pop.append(track["track"]["popularity"])
        counts.append(count)
        count += 1
    
print(f"Song name count: {len(song_name)}")
print(f"Artist name count:{len(artist_name)}")
print(f"Artist pop count:{len(artist_pop)}")
print(f"Artist genre count:{len(genre_list)}")
print(f"Artist album count:{len(album)}")
print(f"Track pop count:{len(track_pop)}")
print(f"Artist name count:{len(song_years)}")
print(f"Position in top 50 count:{len(counts)}")

In [ ]:
top_songs_df = pd.DataFrame({"Year": song_years,
                            "Song Name": song_name, 
                            "Artist": artist_name,
                            "Artist Popularity": artist_pop,
                            "Genre": genre_list,
                            "Album": album,
                            "Song Popularity": track_pop,
                            "Song Position in top 50": counts})
top_songs_df

In [ ]:
top_songs_df['Artist'].value_counts()

In [ ]:
post_malone_df = top_songs_df.loc[(top_songs_df['Artist'] == 'Post Malone') | (top_songs_df['Artist'] == 'Drake')]
post_malone_df

In [ ]:
#scatter = post_malone_df.plot.scatter(x='Year', y = 'Song Position in top 50', color = 'green', ylim=(0,50))
scatter = sns.scatterplot(x="Year", y="Song Position in top 50", data=post_malone_df, hue="Artist")
plt.title('Post Malone compared to Drake Top 50 hits per year')
ax = scatter.axes
ax.invert_yaxis()
plt.savefig('Output/fig9')

In [ ]:
#ho = popularity of a song is influenced by the release year of the song (reject if p_value < 0.5)
#ha = popularity of a song is not influenced by the release year of the song

In [ ]:
#Null hypothesis = Post Malone is in top 10 80% of the time
#popularity of a song is influenced by the release year of the song
top_songs_df.groupby('Year')['Song Popularity'].mean()

In [ ]:
song_populatiry_year1 = top_songs_df[top_songs_df['Year'] == '2017']['Song Popularity']
song_populatiry_year2 = top_songs_df[top_songs_df['Year'] == '2023']['Song Popularity']

In [ ]:
#there is a asignificant difference in the song popularities by post malone and drake

In [ ]:
ttest,pvalue = stats.ttest_ind(song_populatiry_year1, song_populatiry_year2)

In [ ]:
pvalue

In [ ]:
top_songs_df[top_songs_df['Artist']=='Post Malone']['Song Popularity'].mean()

In [ ]:
top_songs_df[top_songs_df['Artist']=='Drake']['Song Popularity'].mean()

In [ ]:
cat1 = top_songs_df[top_songs_df['Artist']=='Post Malone']
cat2 = top_songs_df[top_songs_df['Artist']=='Drake']

ttest_ind(cat1['Song Popularity'], cat2['Song Popularity'])

In [ ]:
top_songs_df.columns

In [ ]:
top_songs_2016_df = top_songs_df.loc[top_songs_df['Year'] == '2016']
top_songs_2017_df = top_songs_df.loc[top_songs_df['Year'] == '2017']
top_songs_2018_df = top_songs_df.loc[top_songs_df['Year'] == '2018']
top_songs_2019_df = top_songs_df.loc[top_songs_df['Year'] == '2019']
top_songs_2020_df = top_songs_df.loc[top_songs_df['Year'] == '2020']
top_songs_2021_df = top_songs_df.loc[top_songs_df['Year'] == '2021']
top_songs_2022_df = top_songs_df.loc[top_songs_df['Year'] == '2022']

In [ ]:
top_10_songs_df = top_songs_df.loc[top_songs_df['Song Position in top 50'] < 11]
top_10_songs_df

In [ ]:
top_10_songs_df.boxplot(by="Year", column="Song Popularity")
plt.title('Boxplot grouped by Year, top 10 Songs Song Popularity')
plt.savefig('Output/fig1')

In [ ]:
top_10_songs_df.plot.scatter(x='Year', y = 'Song Popularity')
plt.savefig('Output/fig2')

In [ ]:
top_songs_df.boxplot(by="Year", column="Song Popularity")
plt.savefig('Output/fig3')

In [ ]:
top_songs_df.plot.scatter(x='Year', y = 'Song Popularity')
plt.savefig('Output/fig4')

In [ ]:
top_songs_df.plot.scatter(x='Artist Popularity', y = 'Song Popularity', color = 'green')
plt.savefig('Output/fig5')

In [ ]:
#null hypothesis: Song popularity and Artist Popularity are the same the majority of the time
song_pop_test = top_songs_df['Song Popularity'].mean()
print(song_pop_test)
artist_pop_test =top_songs_df['Artist Popularity'].mean()
print(artist_pop_test)
#ttest,pvalue = stats.ttest_ind(song_pop_test, artist_pop_test)
#pvalue

In [ ]:
v = top_songs_df['Genre'].value_counts().index
top_genres_df = top_songs_df[top_songs_df['Genre'].isin(top_songs_df['Genre'].value_counts()[top_songs_df['Genre'].value_counts()>1].index)]
#top_genres_df['Genre'].value_counts()
top_genres_df['Genre'].value_counts()[:30].plot(kind='bar')
plt.xlabel('Genres') 
plt.ylabel('Number of Songs')
plt.title('Top 30 Genres from 2016-2023 in Top Hits Playlists')
plt.savefig('Output/fig6')

In [ ]:
artistis_frequency_dict = top_songs_df['Artist'].value_counts()
artists_more_than_two = [k for k,v in artistis_frequency_dict.items() if v>1]
artists_more_than_two_df = top_songs_df[top_songs_df['Artist'].isin(artists_more_than_two)]
artists_more_than_two_df

In [ ]:
plt.figure(figsize=(30,5))
artists_more_than_two_df['Artist'].value_counts().plot.bar()
plt.xlabel('Artists')
plt.ylabel('Times in Top 50')
plt.title("Artists who appeared in Top 50 Hits more than once from 2016-2023")
plt.savefig('Output/fig7')

In [ ]:
#Top Songs From 2016-2022(Based on popularity greater than 85)
top_popularity_songs_df = (top_songs_df.loc[top_songs_df['Song Popularity'] > 85]).sort_values(by=['Song Popularity'], ascending = False)

top_popularity_songs_df.plot.scatter(x='Song Name', y='Song Popularity', title='Songs with popularity greater than 85(2016-2022 Top Hits)',ylabel='Popularity(Spotify Value)', 
                                 rot=90, ylim=(0,100), figsize=(20,5))
plt.savefig('Output/fig8')


In [ ]:
artist_name = 'Drake'
url = 'https://api.spotify.com/v1/search'
headers = {"Authorization": "Bearer " + token}
query = f"?q={artist_name}&type=artist&limit=1"
query_url = url + query
result = get(query_url, headers=headers)
json_result = json.loads(result.content)
json_result['artists']